In [13]:
import csv
import cv2
import numpy as np
%matplotlib inline

lines = []
with open ('./mydata/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
    
images = []
measurements = []

for line in lines:
    source_path = line[0]
    if source_path == 'center':
        continue
    filename = source_path.split('/')[-1]
    current_path = './mydata/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
# Augment data with fliping images and taking opposite sign of the steering measuremnts
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*(-1.0))

X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

In [14]:
from keras.models import Sequential
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Cropping2D
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU

model = Sequential()

# Crop inside the model
model.add(Cropping2D(cropping=((70, 25), (1, 1)), input_shape=(160, 320, 3)))

# Normalize
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160, 320, 3)))

# Add 5 convolutional layers
model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2,2)))
model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2,2)))
model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2,2)))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Conv2D(64, 3, 3, activation='elu'))

# Add a flatten layer
model.add(Flatten())

# Add 3 fully connected layers
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))

# Add a fully connected output layers
model.add(Dense(1))

# Compile and train the model
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)

# Print model summary
print(model.summary)

# Save model
model.save('model.h5')

Train on 3646 samples, validate on 912 samples
Epoch 1/5
3646/3646 [==============================] - 7s - loss: 0.0114 - val_loss: 0.0029
Epoch 2/5
3646/3646 [==============================] - 6s - loss: 0.0042 - val_loss: 0.0036
Epoch 3/5
3646/3646 [==============================] - 6s - loss: 0.0028 - val_loss: 0.0037
Epoch 4/5
3646/3646 [==============================] - 6s - loss: 0.0024 - val_loss: 0.0047
Epoch 5/5
3646/3646 [==============================] - 6s - loss: 0.0016 - val_loss: 0.0029
<bound method Container.summary of <keras.models.Sequential object at 0x7f7a7e7a5c88>>
